In [1]:
#importing the libraries

import re
import string
import datetime as dt
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import nltk
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from time import time, sleep
from wordcloud import WordCloud, STOPWORDS
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.corpus import wordnet
from sklearn.decomposition import LatentDirichletAllocation
from textblob import TextBlob
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from textblob import TextBlob
import numpy as np
from nltk.stem import WordNetLemmatizer
# nltk.download('wordnet')
try:
    nlp = spacy.load("en_core_web_sm")
except:
    import en_core_web_sm
    nlp = en_core_web_sm.load()

In [2]:
#importing the data
AppleTweets = pd.read_csv('data/AppleTweets.csv')
SamsungTweets = pd.read_csv('data/SamsungTweets.csv')
HuaweiTweets = pd.read_csv('data/HuaweiTweets.csv')

In [3]:
#looking into the number of rows and columns of each dataset
print(AppleTweets.shape,SamsungTweets.shape,HuaweiTweets.shape)

(15573, 22) (14841, 21) (14724, 21)


In [4]:
#dropping the unwanted columns
AppleTweets.drop(AppleTweets.columns[AppleTweets.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
AppleTweets.drop(["screen_name", "username","user_id","tweet_id","tweet_url","timestamp_epochs",
                 "text_html","links","has_media","img_urls","video_url",
                 "is_replied","is_reply_to","parent_tweet_id","reply_to_users"], axis = 1, inplace = True) 

SamsungTweets.drop(SamsungTweets.columns[SamsungTweets.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
SamsungTweets.drop(["screen_name", "username","user_id","tweet_id","tweet_url","timestamp_epochs",
                 "text_html","links","has_media","img_urls","video_url",
                 "is_replied","is_reply_to","parent_tweet_id","reply_to_users"], axis = 1, inplace = True) 

HuaweiTweets.drop(HuaweiTweets.columns[HuaweiTweets.columns.str.contains('unnamed',case = False)],axis = 1, inplace = True)
HuaweiTweets.drop(["screen_name", "username","user_id","tweet_id","tweet_url","timestamp_epochs",
                 "text_html","links","has_media","img_urls","video_url",
                 "is_replied","is_reply_to","parent_tweet_id","reply_to_users"], axis = 1, inplace = True) 

print(AppleTweets.shape,SamsungTweets.shape,HuaweiTweets.shape)

(15573, 6) (14841, 6) (14724, 6)


In [5]:
#creating new columns with the name of the each company
AppleTweets['company'] = 'Apple'
SamsungTweets['company'] = 'Samsung'
HuaweiTweets['company'] = 'Huawei' 
#combaning the datasets
Data = AppleTweets.append([SamsungTweets,HuaweiTweets])
Data.reset_index(inplace=True, drop=True)
Data.shape

(45138, 7)

In [6]:
#creating a new copy of the dataset
data = Data.copy()
data.head(10)

,timestamp,text,hashtags,likes,retweets,replies,company
0,2019-12-25 23:59:50,...Thank You! Just a bummer cause he’s had Fit...,[],1,0,1,Apple
1,2019-12-25 23:59:23,@Apple Thanks for the quality products you so...,[],0,0,0,Apple
2,2019-12-25 23:57:25,My @Apple AirPods only worked for 18 months. J...,[],0,0,0,Apple
3,2019-12-25 23:56:09,What I get all the time on iOS13 is this. Star...,[],1,0,0,Apple
4,2019-12-25 23:56:04,@Apple @AppleSupport Why has it taken over 5 h...,[],0,0,1,Apple
5,2019-12-25 23:55:00,NaN,[],1,0,0,Apple
6,2019-12-25 23:53:41,@Apple Why can't I stream audio from my iPhone...,[],0,0,0,Apple
7,2019-12-25 23:53:31,Gotta know why @Apple needs to track and save ...,[],0,0,0,Apple
8,2019-12-25 23:52:41,Opooo HAHAHAHAHAHAHAG,[],1,0,1,Apple
9,2019-12-25 23:52:22,masaya ka?,[],1,0,1,Apple


In [7]:
#Dropping the rows which have empty text
data.drop(index=data.loc[data['text'].isnull()].index, inplace=True)
data.reset_index(inplace=True, drop=True)
data.shape

(44739, 7)

In [8]:
# dropping ALL duplicte values 
data.drop_duplicates(subset ="text", 
                     keep = False, inplace = True)
data.reset_index(inplace=True, drop=True)
data.shape

(39730, 7)

In [9]:
#removing non english words
words = set(nltk.corpus.words.words())
for i in data.index:
    data['text'][i] =  " ".join(w for w in nltk.wordpunct_tokenize(data['text'][i]) if w.lower() in words or not w.isalpha())
    
data.head()

C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


,timestamp,text,hashtags,likes,retweets,replies,company
0,2019-12-25 23:45:56,Does anyone seriously believe that the @ Apple...,[],1,0,0,Apple
1,2019-12-25 23:45:50,My phone itself last night and now half my hav...,[],0,0,1,Apple
2,2019-12-25 23:44:51,"Dear @ Apple , I never thought that I ' d say ...",[],0,0,0,Apple
3,2019-12-25 23:42:55,See needs a new season !,[],0,0,0,Apple
4,2019-12-25 23:42:53,@ Apple ya di con el SE2,[],0,0,0,Apple


In [10]:
# Convert string to list for hashtag column
hashtag_remove_map = dict((x, ' ') for x in ["[", "]", "'", ","])
trantab = str.maketrans(hashtag_remove_map)
temp_hashtag_list = data['hashtags'].values
for i in data.index:
    temp = temp_hashtag_list[i].translate(trantab).split()
    if (len(temp)> 0):
        data['hashtags'][i] = temp
    else:
        data['hashtags'][i] = []

C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [11]:
#Dropping the rows which have empty text
data.drop(index=data.loc[data['text'].isnull()].index, inplace=True)
data.reset_index(inplace=True, drop=True)
data.shape

(39730, 7)

In [12]:
# Apply first round of text cleaning techniques
def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    #get rid of non english charectors
    text = re.sub("([^\x00-\x7F])+","",text)
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [13]:
# Let's take a look at the updated text
data['text'] = pd.DataFrame(data['text'].apply(round1))
data

,timestamp,text,hashtags,likes,retweets,replies,company
0,2019-12-25 23:45:56,does anyone seriously believe that the apple ...,[],1,0,0,Apple
1,2019-12-25 23:45:50,my phone itself last night and now half my hav...,[],0,0,1,Apple
2,2019-12-25 23:44:51,dear apple i never thought that i d say thi...,[],0,0,0,Apple
3,2019-12-25 23:42:55,see needs a new season,[],0,0,0,Apple
4,2019-12-25 23:42:53,apple ya di con el,[],0,0,0,Apple
...,...,...,...,...,...,...,...
39725,2019-12-18 05:59:15,type c pro surface go galaxy tab ...,[],0,0,0,Huawei
39726,2019-12-18 01:33:06,d,[],0,0,0,Huawei
39727,2019-12-17 23:47:06,i just write and consume some content i am th...,[],0,0,1,Huawei
39728,2019-12-17 23:12:26,x pro,[],0,0,2,Huawei


In [14]:
# Applying the second round of text cleaning, Removing punctuations, special characters and numbers
data['text'] = data['text'].str.replace("[^a-zA-Z#]", " ") 
data.shape

(39730, 7)

In [15]:
#Replacing the empty texts with numpy nan and removing them
data['text'].replace('', np.nan, inplace=True)
data.dropna(subset=['text'], inplace=True)
#reseting the database index
data.reset_index(inplace=True, drop=True)
data.shape

(38021, 7)

In [16]:
from nltk.corpus import stopwords 
from nltk.tokenize import word_tokenize 

#tokenizing and removing stop words
stop_words = set(stopwords.words('english')) 
for i in data.index:
    word_tokens = word_tokenize(data['text'][i])
    data['text'][i] = [w for w in word_tokens if not w in stop_words] 
    
data.head()

C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,timestamp,text,hashtags,likes,retweets,replies,company
0,2019-12-25 23:45:56,"[anyone, seriously, believe, apple, timcook, c...",[],1,0,0,Apple
1,2019-12-25 23:45:50,"[phone, last, night, half, phone, apple, hell]",[],0,0,1,Apple
2,2019-12-25 23:44:51,"[dear, apple, never, thought, say, miss, apple...",[],0,0,0,Apple
3,2019-12-25 23:42:55,"[see, needs, new, season]",[],0,0,0,Apple
4,2019-12-25 23:42:53,"[apple, ya, di, con, el]",[],0,0,0,Apple


In [17]:
#  using nltk WordNetLemmatizer function to normalize the tweets
# Text Normalization -reducing terms to their base word(like runs, run, running, ran to base word run)
lemmatizer = WordNetLemmatizer()
data['text'] = data['text'].apply(lambda x: [lemmatizer.lemmatize(i) for i in x])


for i in range(len(data['text'])):
    data['text'][i] = ' '.join(data['text'][i])    

data.head()

C:\Users\DELL\anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,timestamp,text,hashtags,likes,retweets,replies,company
0,2019-12-25 23:45:56,anyone seriously believe apple timcook capable...,[],1,0,0,Apple
1,2019-12-25 23:45:50,phone last night half phone apple hell,[],0,0,1,Apple
2,2019-12-25 23:44:51,dear apple never thought say miss apple piece ...,[],0,0,0,Apple
3,2019-12-25 23:42:55,see need new season,[],0,0,0,Apple
4,2019-12-25 23:42:53,apple ya di con el,[],0,0,0,Apple


In [18]:
from collections import Counter 
  
def remove_duplicates(input): 
  
    # split input string separated by space 
    input = input.split(" ") 
  
    # joins two adjacent elements in iterable way 
    for i in range(0, len(input)): 
        input[i] = "".join(input[i]) 
  
    # now create dictionary using counter method 
    # which will have strings as key and their  
    # frequencies as value 
    UniqW = Counter(input) 
  
    # joins two adjacent elements in iterable way 
    s = " ".join(UniqW.keys()) 
    return s

In [19]:
# remove_duplicates
data['text'] = pd.DataFrame(data['text'].apply(remove_duplicates))

In [20]:
# Removing short words less than length 3 since they are meaningless in the context
data['text'] = data['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
data

,timestamp,text,hashtags,likes,retweets,replies,company
0,2019-12-25 23:45:56,anyone seriously believe apple timcook capable...,[],1,0,0,Apple
1,2019-12-25 23:45:50,phone last night half apple hell,[],0,0,1,Apple
2,2019-12-25 23:44:51,dear apple never thought miss piece garbage,[],0,0,0,Apple
3,2019-12-25 23:42:55,need season,[],0,0,0,Apple
4,2019-12-25 23:42:53,apple,[],0,0,0,Apple
...,...,...,...,...,...,...,...
38016,2019-12-18 05:59:15,type surface galaxy,[],0,0,0,Huawei
38017,2019-12-18 01:33:06,,[],0,0,0,Huawei
38018,2019-12-17 23:47:06,write consume content thinking love think futu...,[],0,0,1,Huawei
38019,2019-12-17 23:12:26,,[],0,0,2,Huawei


In [21]:
#create a new row for tokens
data['tokenizedWords'] = data['text'].apply(lambda x: x.split())

In [22]:
# count the number of words in each doc
word_count_list = np.zeros(data.shape[0], dtype=np.int)
doc_list = data['text'].values
for i in data.index:
    word_count_list[i] = len(doc_list[i].split())

# add the word count as a new column
data['word_count'] = word_count_list

data.head()

,timestamp,text,hashtags,likes,retweets,replies,company,tokenizedWords,word_count
0,2019-12-25 23:45:56,anyone seriously believe apple timcook capable...,[],1,0,0,Apple,"[anyone, seriously, believe, apple, timcook, c...",17
1,2019-12-25 23:45:50,phone last night half apple hell,[],0,0,1,Apple,"[phone, last, night, half, apple, hell]",6
2,2019-12-25 23:44:51,dear apple never thought miss piece garbage,[],0,0,0,Apple,"[dear, apple, never, thought, miss, piece, gar...",7
3,2019-12-25 23:42:55,need season,[],0,0,0,Apple,"[need, season]",2
4,2019-12-25 23:42:53,apple,[],0,0,0,Apple,[apple],1


In [23]:
#removing the rows with 0 word
data = data[data.word_count != 0]
data.reset_index(inplace=True, drop=True)
data.shape

(34647, 9)

In [24]:
def hashtag_remover(df,tag):
    temp = []
    for ind, hashtags in enumerate(df.hashtags):
        if tag in hashtags:
            temp.append(ind)
    df.drop(index=temp,inplace=True)
    df.reset_index(inplace=True, drop=True)
    return df.shape

In [25]:
#removing unrelated rows
hashtag_remover(data,'CarlotaNoEstasSola')


C:\Users\DELL\anaconda3\lib\site-packages\pandas\core\frame.py:3997: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


(34601, 9)

In [26]:
data.to_csv('data/preprocessedData.csv', index=False)